In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))

In [ ]:
library(forcats)
#library(wesanderson)

### 0 - Read exhaustive analysis

In [ ]:
fe <- fread(paste0(SHARE_DIR, "0_run_fe.csv"))

In [ ]:
remove_last <- function(i) i[-length(i)]

In [ ]:
options(repr.plot.height= 6, repr.plot.width = 8)

In [ ]:
base <- 
fe %>% 
 fi(!grepl("geneset_mp", feature)) %>% 
 rw() %>% 
 mu( raw = paste(remove_last(strsplit(feature, "_")[[1]]), collapse = "_")) %>% 
 ug() %>% 
 gb(cohortGo, raw, fisher_pval) %>% 
 mu( rk = row_number(desc(e_nr)), 
     label = factor(ifelse(pr_nr_given_e > pr_nr_overall, 
                           "Non-Response Associated", 
                           "Response Associated"), 
                   levels = c("Response Associated", "Non-Response Associated"))) %>%
 fi(rk == 1) %>% se(-rk) %>% 
 mu(or =  (e_r/e_nr)/(ne_r/ne_nr)) %>% 
 fi(cohortGo != "Pan-Cancer ## Folinic acid ## Platinum ## Pyrimidine (ant)agonist ## Topoisomerase inhibitor") 

base$p_adj_BH <- p.adjust(base$fisher_pval, method = "BH")
base$p_adj_BY <- p.adjust(base$fisher_pval, method = "BY")

In [ ]:
base <-
base %>% 
mu(p_cat = ifelse(fisher_pval < .05, "raw", "non"),
   p_cat = ifelse(p_adj_BH < .05, "BH", p_cat),
   p_cat = ifelse(p_adj_BY < .05, "BY", p_cat))  

In [ ]:
labels <- 
base %>% 
 gb(cohortGo, or > 1) %>% 
 mu(rk = row_number(fisher_pval)) %>%
 fi(rk == 1)

In [ ]:
alphas <- 
c("BY" = 1, "BH" = .2, "raw" = .1, "non" = .01) 

In [ ]:
table( base$p_cat )

In [ ]:
alphas

In [ ]:
base %>% 
 fi(grepl("Pan-Cancer", cohortGo)) %>% 
 ggplot( aes(y = -log10(fisher_pval), x = log(or))) + 
 geom_point() + 
 #scale_alpha_manual( values = alphas ) + 
 facet_wrap(~fct_reorder(cohortGo, fisher_pval, .fun = min), ncol = 3) + 
 labs( y = "-Log 10 (Fisher Exact P-value)", x = "Log Odds Response") + 
 go_theme + 
 ylim(0,11) + xlim(-5,5) + 
 geom_hline(yintercept = -log10(.05), type = "dashed") + 
 theme(legend.position = "none") + 
 geom_text(data = labels %>% fi(grepl("Pan-Cancer", cohortGo)), aes(label = feature), vjust = -1, color = "black", size = 3)